In [1]:
%matplotlib inline
import matplotlib.pyplot as plt


import os
import itertools
import json
import numpy as np
import pandas as pd
import pickle
import requests
import seaborn as sns
import collections
from collections import Counter
import scipy
import time
import copy
from collections import OrderedDict

import matplotlib as mpl
import matplotlib.gridspec as gridspec
from matplotlib.patches import Rectangle
import matplotlib.patches as mpatches

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition.pca import PCA
from sklearn.linear_model import LinearRegression

import nltk
#nltk.download('wordnet')
#nltk.download('stopwords')
#nltk.download('punkt')
#nltk.download('averaged_perceptron_tagger')
import gensim
import re
from fuzzywuzzy import process


/Users/stefgarasto/miniconda3/envs/nlp/lib/python3.6/site-packages/smart_open/ssh.py:34: UserWarning: paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress
  warnings.warn('paramiko missing, opening SSH/SCP/SFTP paths will be disabled.  `pip install paramiko` to suppress')


In [2]:
from utils_nos import nesta_colours, nesta_colours_combos
print(nesta_colours, nesta_colours_combos)

[[1, 0.7215686274509804, 0.09803921568627451], [1, 0, 0.2549019607843137], [0, 0, 0], [1, 0.35294117647058826, 0], [0.6078431372549019, 0, 0.7647058823529411], [0.6470588235294118, 0.5803921568627451, 0.5098039215686274], [0.6274509803921569, 0.5686274509803921, 0.1568627450980392], [0.7686274509803922, 0.6901960784313725, 0], [0.9647058823529412, 0.49411764705882355, 0], [0.7843137254901961, 0.1568627450980392, 0.5725490196078431], [0.23529411764705882, 0.07058823529411765, 0.3215686274509804]] [[0, 1, 2, 3, 4, 5], [0, 6, 7], [1, 3, 8], [4, 9, 10], [8, 5], [1, 11]]


In [3]:
# set up plot style
print(plt.style.available)
plt.style.use(['seaborn-darkgrid','seaborn-poster','ggplot'])

['seaborn-dark', 'seaborn-darkgrid', 'seaborn-ticks', 'fivethirtyeight', 'seaborn-whitegrid', 'classic', '_classic_test', 'fast', 'seaborn-talk', 'seaborn-dark-palette', 'seaborn-bright', 'seaborn-pastel', 'grayscale', 'seaborn-notebook', 'ggplot', 'seaborn-colorblind', 'seaborn-muted', 'seaborn', 'Solarize_Light2', 'seaborn-paper', 'bmh', 'tableau-colorblind10', 'seaborn-white', 'dark_background', 'seaborn-poster', 'seaborn-deep']


In [4]:
# flatten lists of lists
def flatten_lol(t):
    return list(itertools.chain.from_iterable(t))
flatten_lol([[1,2],[3],[4,5,6]])


[1, 2, 3, 4, 5, 6]

In [5]:
#These two functions are useful for analysing bi and tri-grams with w2v models in gensim

def convert_to_undersc(skill):
    '''
    convert spaces in skill phrases into underscores to use with trained
    w2v model.
    '''
    if len(skill.split(' ')) >1:
        new_i = '-'.join(skill.split(' '))
    else:
        new_i = skill
    return(new_i)

def convert_from_undersc(skill):
    '''
    convert underscores between terms in skill phrases back to spaces.
    '''
    if len(skill.split('_')) >1:
        new_i = ' '.join(skill.split('_'))
    else:
        new_i = skill
    return(new_i)


In [6]:
#A few functions for tyding up text
def tag_for_lemmatise(s):
    pos_to_wornet_dict = {
        'JJ': 'a',
        'JJR': 'a',
        'JJS': 'a',
        'RB': 'r',
        'RBR': 'r',
        'RBS': 'r',
        'NN': 'n',
        'NNP': 'n',
        'NNS': 'n',
        'NNPS': 'n',
        'VB': 'v',
        'VBG': 'v',
        'VBD': 'v',
        'VBN': 'v',
        'VBP': 'v',
        'VBZ': 'v',
    }
    try:
        return pos_to_wornet_dict[nltk.pos_tag([s])[0][1]]
    except:
        return 'n'
    
def lemmatise(title_terms):
    """
    Takes list as input.
    Removes suffixes if the new words exists in the nltk dictionary.
    The purpose of the function is to convert plural forms into singular.
    Allows some nouns to remain in plural form (the to_keep_asis is manually curated).
    Returns a list.
    >>> lemmatise(['teachers'])
    ['teacher']
    >>> lemmatise(['analytics'])
    ['analytics']
    """
    keep_asis = ['sales', 'years', 'goods', 'operations', 'systems',
                    'communications', 'events', 'loans', 'grounds',
                    'lettings', 'claims', 'accounts', 'relations',
                    'complaints', 'services']
    wnl = nltk.WordNetLemmatizer()
    processed_terms = [wnl.lemmatize(i) if i not in keep_asis else i for i in title_terms]
    #processed_terms = [wnl.lemmatize(i, pos = tag_for_lemmatise(i)) 
    #            if i not in keep_asis else i for i in title_terms]
    return processed_terms

def lemmatise_with_pos(title_terms):
    """
    Takes list as input.
    Removes suffixes if the new words exists in the nltk dictionary.
    The purpose of the function is to convert plural forms into singular.
    Allows some nouns to remain in plural form (the to_keep_asis is manually curated).
    Returns a list.
    >>> lemmatise(['teachers'])
    ['teacher']
    >>> lemmatise(['analytics'])
    ['analytics']
    """
    pos_to_wornet_dict = {
        'JJ': 'a',
        'JJR': 'a',
        'JJS': 'a',
        'RB': 'r',
        'RBR': 'r',
        'RBS': 'r',
        'NN': 'n',
        'NNP': 'n',
        'NNS': 'n',
        'NNPS': 'n',
        'VB': 'v',
        'VBG': 'v',
        'VBD': 'v',
        'VBN': 'v',
        'VBP': 'v',
        'VBZ': 'v',
    }
    keep_asis = ['sales', 'years', 'goods', 'operations', 'systems',
                    'communications', 'events', 'loans', 'grounds',
                    'lettings', 'claims', 'accounts', 'relations',
                    'complaints', 'services']
    wnl = nltk.WordNetLemmatizer()
    processed_terms = [wnl.lemmatize(i, pos_to_wornet_dict[p]) if i not in keep_asis else i for i,p in title_terms]
    #processed_terms = [wnl.lemmatize(i, pos = tag_for_lemmatise(i)) 
    #            if i not in keep_asis else i for i in title_terms]
    return processed_terms

def lemmatise_pruned(x, pofs = 'nv'):
    if pofs == 'nv':
        tags = [(t,p) for t,p in x if p[:1] in ['V','N']]
    elif pofs == 'n':
        tags = [(t,p) for t,p in x if p[:1] in ['N']]
    else:
        raise ValueError
    return lemmatise_with_pos(tags)

def remove_digits(s):
    """
    Takes a string as input.
    Removes digits in a string.
    Returns a string.
    >>> remove_digits('2 recruitment consultants')
    ' recruitment consultants'
    """
    result = ''.join(i for i in s if not i.isdigit())
    return result

def remove_list_enumeration(s):
    '''
    This is a specific requirement of the NOS that comes from
    the presence of lists enumerated by strings like K+number
    or P+number. Therefore, after "lowerising" and removing 
    digits, I look for and remove strings like "k " and "p "
    '''
    result = re.sub('( k )+',' ',s)
    result = re.sub('( p )+', ' ', result)
    # it might not be necessary if I add 'k' and 'p' to stopwords
    return result

select_punct = set('!"#$%&\()*+,-./:;<=>?@[\\]^_`{|}~') #only removed "'"
extra_chars = set('–-•’”“µ¾âãéˆﬁ[€™¢±ï…˜')
all_select_chars = select_punct.union(extra_chars)
def replace_punctuation(s):
    """
    Takes string as input.
    Removes punctuation from a string if the character is in select_punct.
    Returns a string.
   >>> replace_punctuation('sales executives/ - london')
   'sales executives   london'
    """
    for i in set(all_select_chars): #set(select_punct):
        if i in s:
            s = s.replace(i, ' ')
    return s

def tidy_desc(desc):
    clean_data = desc.replace('\r\n', '').replace('\xa0', '')
    nodigits = remove_digits(clean_data.lower())
    nopunct = replace_punctuation(nodigits)
    #nopunct = remove_list_enumeration(nopunct)
    lemm = lemmatise(nopunct.split())
    return ' '.join(lemm)

def tokenize(text):
    """
    Takes string as input.
    Returns list of tokens. The function is used as an argument for
    TfidfVectorizer.
    >>> tokenize('some job title')
    ['some', 'job', 'title']
    """
    tokens = nltk.word_tokenize(text)
    return tokens

def tokenize_asis(some_list):
    """
    Takes list as input.
    Returns the list with elements converted to lower case. The function is 
    used as an argument for TfidfVectorizer.
    
    In [57]: tokenize(['Accounting', 'Microsoft Excel'])
    Out[57]: ['accounting', 'microsoft excel']
    """
    tokens = [elem.lower() for elem in some_list]
    return tokens

In [7]:
#This set of functions is useful for identifying terms with highest tf-idf weights 
#in a single document or set of documents

def top_tfidf_feats(row, features, top_n=25):
    ''' Get top n tfidf values in row and return them with their corresponding 
        feature names.'''
    topn_ids = np.argsort(row)[::-1][:top_n]
    top_feats = [(features[i], row[i]) for i in topn_ids]
    df = pd.DataFrame(top_feats)
    df.columns = ['feature', 'tfidf']
    return df

def top_mean_feats(Xtr, features, grp_ids=None, min_tfidf=0.1, top_n=25, sparse_output = False):
    ''' Return the top n features that on average are most important 
        amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    if sparse_output:
        return scipy.sparse.csr_matrix(top_tfidf_feats(tfidf_means, features, top_n))
    else:
        return top_tfidf_feats(tfidf_means, features, top_n)

def all_mean_feats(Xtr, grp_ids=None, min_tfidf=0.1):
    ''' Return the average
        amongst documents in rows
        indentified by indices in grp_ids. '''
    if grp_ids:
        D = Xtr[grp_ids].toarray()
    else:
        D = Xtr.toarray()

    D[D < min_tfidf] = 0
    tfidf_means = np.mean(D, axis=0)
    return tfidf_means

def get_top_words_weights(desc, feature_names, vect, n = 25):
    response = vect.transform(desc)
    words = top_mean_feats(response, feature_names, grp_ids = None, top_n = n)
    return words

def get_mean_tfidf(desc, vect):
    response = vect.transform(desc)
    tfidf_values = all_mean_feats(response, grp_ids = None)
    return tfidf_values

def get_top_words(desc, feature_names, vect, n = 25):
    response = vect.transform(desc)
    words = top_mean_feats(response, feature_names, grp_ids = None, top_n = n)
    return words['feature'].values

In [8]:
def print_elapsed(t0_local, task = 'current task'):
    print('Done with {}. Elapsed time: {:4f}'.format(task,time.time()-t0_local))
    

In [9]:
# manually remove "k"s and "p"s from the pruned columns
def remove_pk(x):
    return [t for t in x if t not in ['k','p']]
#df_nos['pruned'] = df_nos['pruned'].map(remove_pk)

### Define more functions on how to create the TfIdf vectoriser and matrix

In [10]:
# First, create your TFidfVectorizer model. This doesn't depend on whether it's used on suites or NOS. However,
# it does require that the docs collection is already given as a collection of tokens (tokenizer=tokenize_asis)

#Since we now have not just long strings in our documents, but lists of terms, we will use a different tokenizer
def define_tfidf(params, stopwords):
    if params['ngrams'] == 'bi':
        tfidf = TfidfVectorizer(tokenizer=tokenize_asis,
                                lowercase = False,
                                stop_words=stopwords,
                                ngram_range=(1,2), 
                                max_df = params['tfidf_max'], 
                                min_df = params['tfidf_min'])
    elif params['ngrams'] == 'tri':
        tfidf = TfidfVectorizer(tokenizer=tokenize_asis,
                                lowercase = False,
                                stop_words=stopwords,
                                ngram_range=(1,3), 
                                max_df = params['tfidf_max'], 
                                min_df = params['tfidf_min'])
    else:
        # unigrams is the default
        tfidf = TfidfVectorizer(tokenizer=tokenize_asis,
                                lowercase = False,
                                stop_words=stopwords,
                                max_df = params['tfidf_max'], 
                                min_df = params['tfidf_min'])
    return tfidf


In [11]:
# now, collect the text to transform
def combine_nos_text(df_nos, col = 'pruned'):
    all_joint_tokens = []
    # group by suites and concatenate all docs in it
    row_names = []
    for name, group in df_nos.groupby('One_suite'):
        row_names.append(name)
        joint_tokens = []
        for idoc in group[col].index:
            joint_tokens += group[col].loc[idoc]
        all_joint_tokens.append(joint_tokens)
    # return a dataframe
    return pd.DataFrame({'tokens': all_joint_tokens}, index = row_names)

def get_tfidf_matrix(params, df_nos, tfidf, col = 'pruned'):
    # Note: this can simply be used to get the tfidf transform, by setting bywhich=docs and any mode
    t0 = time.time()
    # first, get the dataframe of tokens
    if params['bywhich'] == 'docs':
        textfortoken = df_nos[col]
        
    elif params['bywhich'] == 'suites':
        if params['mode'] == 'meantfidf':
            textfortoken = df_nos[col]
                
        elif params['mode'] == 'combinedtfidf':
            # note that this is the only case where the tfidf min and max are computed considering the number of 
            # suites as the number of elements in the collection.
            # TODO: allow for the alternative case, where the transform is computed on individual NOS and then 
            # applied to the joint tokens
            textfortoken = combine_nos_text(df_nos, col)['tokens']
    
    # apply tfidf transform to the tokenised text
    tfidfm = tfidf.fit_transform(textfortoken)
    
    feature_names = tfidf.get_feature_names()
    
    # if the average is needed, compute it and overwrite the matrix. Note that the step above is still needed to
    # initialise the tfidf transform with the proper features and stopwords
    if (params['bywhich'] == 'suites') and (params['mode'] =='meantfidf'):
        row_names = df_nos['One_suite'].value_counts().index.values
        tfidfm = scipy.sparse.lil_matrix(np.zeros((len(row_names),len(feature_names)), dtype = np.float32))
        for name, group in df_nos.groupby('One_suite'):
            tmp = get_mean_tfidf(group['pruned'], tfidf)
            tfidfm[igroup] = tmp

    print_elapsed(t0, 'computing the tfidf matrix')
    return tfidfm, feature_names, tfidf, textfortoken


## Relationships between standards

In [12]:
from scipy.cluster.hierarchy import ward, dendrogram
from scipy.spatial import distance
from scipy.cluster.hierarchy import cophenet
from scipy.cluster.hierarchy import fcluster 


In [14]:
#We calculate cosine distance between tf-idf vectors of the documents

def do_hierarch_clustering(tfidfm, method='average', metric = 'cosine', DOPLOTS = True):
    t0 = time.time()
    N2 = 11914
    N = 400 #400*400 = 160000 distance calls per second. For N=21500 -- > 462250000 calls --> 2900*160000 calls 
    # --> I'm guessing 2900 seconds = 48 minutes (I think it's likely to be more actually)
    # 4000*4000 takes approximately 110 seconds. It's double for the cophenet. So, for N=22500, the three functions 
    # together will take approx 4 hours (I'll do it tonight)

    try:
        distances = distance.pdist(tfidfm.todense(), metric = metric) # + np.random.randn(N,N2), metric = 'cosine')
        sparse_flag = True
    except:
        distances = distance.pdist(tfidfm, metric = 'cosine')
        sparse_flag = False
    print_elapsed(t0, 'calculating cosine distances of tfidf vectors')

    #We then build linkage matrix using the distances and specifying the method. For euclidean distances typically
    # 'Ward' produces best results. For cosine we can only use 'average' and 'single'.
    linkage_matrix = scipy.cluster.hierarchy.linkage(distances,
                                                     method = method,
                                                     metric = metric)
    print_elapsed(t0, 'hierarchical clustering of cosine distances')
    #We can test how well the groupings reflect actual distances. If c > 0.75 this is considered to be sufficiently
    #good representation
    if sparse_flag:
        c, coph_dists = cophenet(linkage_matrix, 
                             distance.pdist(tfidfm.todense(), metric = 'cosine'))
    else:
        c, coph_dists = cophenet(linkage_matrix, 
                             distance.pdist(tfidfm, metric = 'cosine'))

    print_elapsed(t0, 'computing the cophenetic correlation')

    if DOPLOTS:
        fig, ax =plt.subplots(figsize = (5,5))
        plt.imshow(scipy.spatial.distance.squareform(distances))
        plt.title('cosine distances between suites')
        plt.colorbar()

        fig, ax = plt.subplots(figsize = (5,5))
        tmp = plt.imshow(scipy.spatial.distance.squareform(coph_dists))
        plt.colorbar()
    print('The cophenetic coefficient is {:.4f}'.format(c))
    return distances, linkage_matrix, c, coph_dists



## Choosing parameters for features extraction

ngrams : uni/bi/tri

tfidf thresholds: min and max percentage

which parts of speech were selected before

whether we are working at the level of suites or of invidual NOS, and how we aggregate NOS to form the suit level


## Perform hierarchical clustering on all NOS from one super-suite at a time

In [15]:
def assign_supersuite(x):
    for supersuite in all_match_names.keys():
        if x in all_match_names[supersuite]:
            return supersuite.lower()
    # if no match has been found
    return 'other'

def adjustsoccode(x):
    y = re.findall(r"[\d']+", str(x))
    if len(y):
        return y[0][1:-1]
    else:
        return np.nan

def extract2digits(x):
    if isinstance(x,str):
        try:
            return float(x[:2])
        except:
            return np.nan
    else:
        return np.nan
    
def extract3digits(x):
    if isinstance(x,str):
        try:
            return float(x[:3])
        except:
            return np.nan
    else:
        return np.nan
    
def extract1digits(x):
    if isinstance(x,str):
        try:
            return float(x[:1])
        except:
            return np.nan
    else:
        return np.nan

def extract4digits(x):
    if isinstance(x,str):
        try:
            return float(x)
        except:
            return np.nan
    else:
        return np.nan

In [16]:
def prep_for_gensim(list_of_terms, some_model, weights = None):
    # replace space with underscore
    new_terms = [convert_to_undersc(elem) for elem in list_of_terms]
    # check if each element in the list is in the model
    is_in = [elem for elem in new_terms if elem in some_model]
    # also check the weights
    if weights:
        weights_in = [weights[ix] for ix,elem in enumerate(new_terms) 
                        if elem in some_model]
    # only return the element in the model
    return is_in, weights_in

In [17]:
def get_mean_vec(skill_list, model, weights= None):
    if not weights:
        weights = np.ones(len(skill_list))
    skill_list_conv = [convert_to_undersc(elem) for elem in skill_list]
    wvector_list = [model[elem]*weights[ix] for ix,elem in enumerate(skill_list_conv) 
                if elem in model]
    vector_list = [model[elem] for ix,elem in enumerate(skill_list_conv) 
                if elem in model]
    vec_array = np.asarray(vector_list)
    wvec_array = np.asarray(wvector_list)
    avg_vec = np.mean(wvec_array, axis=0)
    return avg_vec, vec_array

In [18]:
def select_subdf(SELECT_MODE, clusters2use, suites_clusters, df_nos_select):
    if isinstance(SELECT_MODE, str):
        tmp_dict = {'engineering': 'Engineering', 'management': 'Management',
                    'financialservices': 'Financial services', 
                    'construction': 'Construction'}
        # select NOS from super suite
        cluster_name = SELECT_MODE
        cluster_name_save = cluster_name
        cluster_name_figs = tmp_dict[SELECT_MODE]
        subset_nos = df_nos_select[df_nos_select['supersuite']== SELECT_MODE]
    elif isinstance(SELECT_MODE, int):
        cluster_name = clusters2use[SELECT_MODE][1]
        cluster_name_save = cluster_name.replace(' ','_')
        cluster_name_figs = cluster_name.capitalize()
        suites2use = list(suites_clusters[suites_clusters['hierarchical'].map(
                lambda x: x in clusters2use[SELECT_MODE][0])]['Suite_names'].values)
        subset_nos = df_nos_select[df_nos_select['One_suite'].map(
                lambda x: x in suites2use)]
    print('Number of NOS selected: ', len(subset_nos))
    #print(subset_nos.columns)
    
    #%
    # only select those engineering nos with SOC codes
    nosoc = subset_nos['SOC4'].isnull()
    print('percentage of nos without SOC codes: ', nosoc.sum()/len(nosoc))
    if (nosoc.sum())/len(nosoc)<0.9:
        final_nos = subset_nos[~nosoc] #np.isnan(engineering_nos['SOC4'])]
    else:
        final_nos = subset_nos
    final_groups = final_nos.groupby(by = 'One_suite')
    larger_suites = []
    all_lengths = final_groups.agg(len)['NOS Title'].values
    all_lengths[::-1].sort()
    print('Number of NOS in suites belonging to this cluster: ',all_lengths)
    #th_supers = ['engineering': 40, 'financialservices': ]
    for name, group in final_groups:
        if isinstance(SELECT_MODE, int):
            larger_suites.append(name)
        elif len(group)> all_lengths[15]:#th_supers[SELECT_MODE]:
            #print(name, len(group))
            larger_suites.append(name)

    return final_nos, final_groups, larger_suites, cluster_name,  \
                    cluster_name_save, cluster_name_figs


### Main part: setup
the next cell is the setup and it should be the same as the script to detect levels via GMM

In [19]:
WHICH_GLOVE = 'glove.6B.100d' #'glove.6B.100d' #'glove.840B.300d', 
#glove.twitter.27B.100d

glove_dir = '/Users/stefgarasto/Local-Data/wordvecs/'


# In[3]:

# set up plot style
print(plt.style.available)
plt.style.use(['seaborn-darkgrid','seaborn-poster','ggplot'])
pofs = 'nv'
output_dir = '/Users/stefgarasto/Google Drive/Documents/results/NOS/nlp_analysis/'
output_dir += 'nos_clusters_{}_final_no_dropped'.format(pofs)
print(output_dir)
lookup_dir = '/Users/stefgarasto/Google Drive/Documents/results/NOS/extracted/'

#Loading a pre-trained glove model into gensim
# model should have already been loaded in bg_load_prepare_and_run. 
# If not, load it here    

LOADGLOVE = True
if LOADGLOVE:
    print('Loading glove model')
    t0 = time.time()
    # load the glove model
    model = gensim.models.KeyedVectors.load_word2vec_format\
    (os.path.join(glove_dir, 'word2vec.{}.txt'.format(WHICH_GLOVE)))
    #model = api.load("glove-wiki-gigaword-100")  # load pre-trained word-vectors
    # from gensim-data
    #model = Word2Vec(common_texts, size=100, window=5, min_count=1, workers=4)
    #word_vectors = model.wv
    print_elapsed(t0, 'loading the glove model')

    vector_matrix = model.vectors
    list_of_terms = model.index2word

    lookup_terms = [convert_from_undersc(elem) for elem in list_of_terms]




['seaborn-dark', 'seaborn-darkgrid', 'seaborn-ticks', 'fivethirtyeight', 'seaborn-whitegrid', 'classic', '_classic_test', 'fast', 'seaborn-talk', 'seaborn-dark-palette', 'seaborn-bright', 'seaborn-pastel', 'grayscale', 'seaborn-notebook', 'ggplot', 'seaborn-colorblind', 'seaborn-muted', 'seaborn', 'Solarize_Light2', 'seaborn-paper', 'bmh', 'tableau-colorblind10', 'seaborn-white', 'dark_background', 'seaborn-poster', 'seaborn-deep']
/Users/stefgarasto/Google Drive/Documents/results/NOS/nlp_analysis/nos_clusters_nv_final_no_dropped
Loading glove model
Done with loading the glove model. Elapsed time: 38.589213


In [20]:
#%%
'''  Start of the main script   '''

#%% set up main parameters
#from set_params_thematic_groups import qualifier, qualifier0, pofs, WHICH_GLOVE, 
#from set_params_thematic_groups import glove_dir, paramsn
qualifier = 'postjoining_final_no_dropped'
qualifier0 = 'postjoining_final_no_dropped'




#Get the NOS data 
df_nos = pd.read_pickle(lookup_dir + 'all_nos_input_for_nlp_{}.zip'.format(qualifier0))

# load the cleaned and tokenised dataset
df_nos = df_nos.join(pd.read_pickle(lookup_dir + 
                    'all_nos_input_for_nlp_{}_pruned_{}.zip'.format(qualifier,pofs)))

# remove p and k
df_nos['pruned'] = df_nos['pruned'].map(remove_pk)
print('Done with loading the dataset')


# Load stopwords
with open(lookup_dir + 'stopwords_for_nos_{}_{}.pickle'.format(qualifier,pofs),'rb') as f:
    stopwords0, no_idea_why_here_stopwords, more_stopwords = pickle.load(f)
stopwords = stopwords0 + no_idea_why_here_stopwords 
stopwords += tuple(['¤', '¨', 'μ', 'บ', 'ย', 'ᶟ', '‰', '©', 'ƒ', '°', '„'])
stopwords0 += tuple(['¤', '¨', 'μ', 'บ', 'ย', 'ᶟ', '‰', '©', 'ƒ', '°', '„',
                     "'m", "'re", '£','—','‚°','●'])
stopwords0 += tuple(set(list(df_nos['Developed By'])))
stopwords0 += tuple(['cosvr','unit','standard','sfl','paramount','tp','il','al','ad','hoc',
                    'lanleo','ireland','something'])


# In[20]:


# create another column where the texts are lemmatised properly
t0 = time.time()
df_nos['pruned_lemmas'] = df_nos['tagged_tokens'].map(lambda x: lemmatise_pruned(x,pofs))
print(time.time()-t0)


# ### Only keep NOS from a super-suite

# In[21]:


super_suites_files=  ''.join(['/Users/stefgarasto/Google Drive/Documents/data/',
                              'NOS_meta_data/NOS_Suite_Priority.xlsx'])
super_suites_names = ['Engineering','Management','FinancialServices','Construction']
all_super_suites = {}
for which_super_suite in super_suites_names:
    all_super_suites[which_super_suite] = pd.read_excel(super_suites_files, 
                    sheet_name = which_super_suite)
    all_super_suites[which_super_suite]['NOS Suite name'] = all_super_suites[
        which_super_suite]['NOS Suite name'].map(
        lambda x: x.replace('(','').replace('(','').replace('&','and').strip().lower())


# In[22]:


standard_labels = list(df_nos.groupby('One_suite').groups.keys())
all_matches = {}
all_match_names = {}
#match_name = []
for which_super_suite in super_suites_names:
    all_matches[which_super_suite] = []
    for suite in all_super_suites[which_super_suite]['NOS Suite name'].values:
        # do manually some selected suites
        if 'insurance claims' in suite:
            tmp = standard_labels.index('general insurance')
            all_matches[which_super_suite].append(tmp)
            continue
        # for the "management and leadership marketing 2013" both marketing 
        # and marketing 2013 would fit,
        # but I'm only taking the latter
        # find a fuzzy match between 
        out = process.extract(suite, standard_labels, limit=3)
        if len(out) and out[0][1]>89:
            # note: most of them are above 96% similarity (only one is 90%)
            tmp = standard_labels.index(out[0][0])
            #print(suite, out[0])
            if tmp not in all_matches[which_super_suite]:
                all_matches[which_super_suite].append(tmp)
            else:
                if suite == 'installing domestic fascia, soffit, and bargeboards':
                    # this suite is kind of a duplicate - I aggregated it in my suites list
                    continue
                tmp = standard_labels.index(out[2][0])
                all_matches[which_super_suite].append(tmp)
                print(out[0][0],',',out[1][0],',',out[2][0],',',suite)
        else:
            print(suite, ' not found')
            print(out)
            print('\n')
    print(len(all_matches[which_super_suite]),len(all_super_suites[which_super_suite]))
    all_match_names[which_super_suite] = [standard_labels[t] 
                    for t in all_matches[which_super_suite]]


# In[32]:
# assign supersuite and SOC codes
df_nos['supersuite'] = df_nos['One_suite'].apply(assign_supersuite)
# extract 2 digit soc
df_nos['SOC4str'] = df_nos['Clean SOC Code'].map(adjustsoccode)
df_nos['SOC1'] = df_nos['SOC4str'].map(extract1digits)
df_nos['SOC2'] = df_nos['SOC4str'].map(extract2digits)
df_nos['SOC3'] = df_nos['SOC4str'].map(extract3digits)
df_nos['SOC4'] = df_nos['SOC4str'].map(extract4digits)
print(df_nos['supersuite'].value_counts())


# In[38]:


# select NOS in super-suites of interest
df_nos_select = df_nos[~(df_nos['supersuite']=='other')]
print('Nb of NOS in all supersuites' ,len(df_nos_select))


#%%
'''
# ## Get raw data and tokenize

# ## Choosing parameters for features extraction
# 
# ngrams : uni/bi/tri
# 
# tfidf thresholds: min and max percentage
# 
# which parts of speech were selected before
# 
# whether we are working at the level of suites or of invidual NOS, 
# and how we aggregate NOS to form the suit level
# 
'''

#

'''
# First, create your TFidfVectorizer model. This doesn't depend on whether 
it's used on suites or NOS. However,
it does require that the docs collection is already given as a collection of
tokens (tokenizer=tokenize_asis)

#Since we now have not just long strings in our documents, but lists of terms, 
we will use a different tokenizer
'''

paramsn = {}
paramsn['ngrams'] = 'uni'
paramsn['pofs'] = pofs #'nv'
paramsn['tfidf_min'] = 3
paramsn['tfidf_max'] = 0.5

paramsn['bywhich'] = 'docs' #'docs' #'suites'
paramsn['mode'] = 'tfidf' #'tfidf' #'meantfidf' #'combinedtfidf' #'meantfidf'
# define the transform: this one can easily be the same for both 
# keywords and the clustering
tfidf_n = define_tfidf(paramsn, stopwords0)

# get the transform from the whole NOS corpus
FULL_CORPUS = False
if FULL_CORPUS:
    _, feature_names_n, tfidf_n, _ = get_tfidf_matrix(
            paramsn, df_nos, tfidf_n, col = 'pruned_lemmas')
else:
    df_nos_select = df_nos[df_nos['supersuite']=='engineering']
    _, feature_names_n, tfidf_n, _ = get_tfidf_matrix(
            paramsn, df_nos_select, tfidf_n, col = 'pruned_lemmas')


print('Number of features: {}'.format(len(feature_names_n)))
N = 2000
print('Some features:')
print(feature_names_n[N:N+100:3])
print('*'*70)



# In[77]:

# first transform via tfidf all the NOS in one supersuite because you need the top keywords
textfortoken = df_nos_select['pruned_lemmas']
tfidfm = tfidf_n.transform(textfortoken)


        
        

Done with loading the dataset
48.87467122077942
83 83
management and leadership , marketing , marketing 2013 , management and leadership marketing 2013)
59 59
29 29
75 76
other                16350
engineering           3038
construction          1394
management            1376
financialservices      459
Name: supersuite, dtype: int64
Nb of NOS in all supersuites 6267


/Users/stefgarasto/miniconda3/envs/nlp/lib/python3.6/site-packages/sklearn/feature_extraction/text.py:301: UserWarning: Your stop_words may be inconsistent with your preprocessing. Tokenizing the stop words generated tokens ['1', '@'] not in stop_words.
  'stop_words.' % sorted(inconsistent))


Done with computing the tfidf matrix. Elapsed time: 0.940246
Number of features: 4847
Some features:
['half', 'halve', 'hand', 'handbook', 'handing', 'handover', 'hangar', 'harbour', 'hardening', 'hardware', 'harmful', 'harvest', 'haswa', 'hatching', 'header', 'headphone', 'hear', 'heat', 'heatproof', 'heaving', 'helicopter', 'helmet', 'hexagonal', 'hfo', 'hierarchy', 'hindrance', 'historic', 'hob', 'holder', 'holiday', 'homing', 'honeycomb', 'hood', 'hopper']
**********************************************************************


In [21]:
SAVEKW= False
top_terms_dict = {}
top_weights_dict = {}
top_keywords_dict = {}
#for name, group in ifa_df.groupby('Route'):
igroup = 0
n_keywords =[]
n_repeated = []
#top_terms = {}
t0 = time.time()
tfidfm_dense = tfidfm.todense()
for ix,name in enumerate(df_nos_select.index):
    #top_terms = get_top_words(df_nos_select.loc[name]['pruned'], feature_names_n, tfidf, n = 20)
    top_ngrams = np.argsort(tfidfm_dense[ix,:])
    top_ngrams = top_ngrams.tolist()[0][-20:]
    top_ngrams = top_ngrams[::-1]
    # only retain the ones with non zero features
    top_ngrams = [elem for elem in top_ngrams if tfidfm_dense[ix,elem]>0]
    top_weights = [tfidfm_dense[ix,elem] for elem in top_ngrams]
    top_features = [feature_names_n[elem] for elem in top_ngrams]
    top_terms_dict[name] = {}
    top_terms_dict[name] = top_features
    top_weights_dict[name] = {}
    top_weights_dict[name] = top_weights
    if np.random.randn(1)>3:
        print(name, df_nos_select.loc[name]['NOS Title'])
        print(top_features) #, top_keywords)
        print('**************************************')
    if ix % 1000 == 999:
        print('Got to NOS nb {}. Total time elapsed: {:.4f} s'.format(ix,time.time()-t0))

# save them all as csv
if SAVEKW:
    pd.DataFrame.from_dict(top_terms_dict, orient = 'index').to_csv(output_dir +
                        '/NOS_from_supersuites_top_terms_{}_{}.csv'.format(qualifier,pofs))
    pd.DataFrame.from_dict(top_weights_dict, orient = 'index').to_csv(output_dir +
                '/NOS_from_supersuites_top_terms_weights_{}_{}.csv'.format(qualifier,pofs))
tfidfm_dense = None


# In[78]:


# just to check results
'''
print(list(top_terms_dict.keys())[885:887])
top_terms_weights = get_top_words_weights([df_nos_select.iloc[0]['pruned_lemmas']], feature_names_n, tfidf, n = 20)
print(top_terms_weights.sort_values(by = 'tfidf', ascending = False).head(n=20))
'''
# note that the get_top_words_weights function is probably wrong - but it doesn't matter now
print('not now')


# remove top terms that are not in the chosen gensim model
new_top_terms_dict = {}
new_top_weights_dict = {}
for k,v in top_terms_dict.items():
    # check if the top terms for each document are in the gensim model
    new_top_terms, weights = prep_for_gensim(v, model, weights = top_weights_dict[k])
    # only retains the ones in the model
    new_top_terms_dict[k] = new_top_terms
    new_top_weights_dict[k] = weights
    if np.random.randn(1)>3.5:
        print(k, new_top_terms, len(new_top_terms), len(v))
        
'''
Compute average word embedding for each NOS
'''
t0 = time.time()
we_p_st = {}
counter = 0
for ix,k in enumerate(new_top_terms_dict):
    avg_features, _  = get_mean_vec(new_top_terms_dict[k], model, 
                                                  weights = new_top_weights_dict[k])
    we_p_st[k] = avg_features

# In[ ]:


# add the best clusters to the nos dataframe
tmp = pd.DataFrame.from_dict(we_p_st, orient = 'index')
all_we_nb = range(len(avg_features))
all_we_cols = []
all_we_changes = {}
for col in all_we_nb:
    all_we_changes[col] = 'weighted_avg_we{}'.format(col)
    all_we_cols.append('weighted_avg_we{}'.format(col))

tmp = tmp.rename(columns = all_we_changes) #{col: 'weighted_avg_we{}'.format(col)})
if 'weighted_avg_we1' in df_nos_select.columns:
    df_nos_select = df_nos[df_nos['supersuite']=='engineering']
df_nos_select = df_nos_select.join(tmp) #[all_we_cols] = tmp[all_we_cols]
print_elapsed(t0, 'computing average word embedding')

for ix in range(20):
    nos = df_nos_select.index[ix]
    print(nos)
    print(df_nos_select.loc[nos]['NOS Title'])
    print(new_top_terms_dict[nos])
    print(model.similar_by_vector(we_p_st[nos]))
    print()

Got to NOS nb 999. Total time elapsed: 0.3645 s
semaut3083.pdf producing motorsport composite assemblies
['assembly', 'composite', 'motorsport', 'produce', 'join', 'joining', 'mould', 'duct', 'fairings', 'panel', 'floor', 'moulding', 'body', 'adhesive', 'rivet', 'trial', 'incorporate', 'shroud', 'tub', 'wing']
**************************************
sembit33.pdf applying central limit theorem and confidence intervals
['confidence', 'interval', 'mean', 'calculate', 'limit', 'equation', 'cpk', 'cp', 'error', 'population', 'distribute', 'utilise', 'deviation', 'estimate', 'size', 'highlight', 'measurement', 'finding', 'data', 'improvement']
**************************************
semfwe344.pdf forming pipework by machine bending
['bend', 'bending', 'pipe', 'machine', 'form', 'forming', 'produce', 'pipework', 'section', 'angle', 'bridge', 'operate', 'expansion', 'diameter', 'template', 'deformation', 'perform', 'power', 'offset', 'size']
**************************************
Got to NOS nb 1

[('maintenance', 0.7866942882537842), ('system', 0.7516964673995972), ('treatment', 0.7326031923294067), ('facilities', 0.7323371171951294), ('facility', 0.7322425842285156), ('equipment', 0.7315930724143982), ('systems', 0.7310415506362915), ('use', 0.7290605902671814), ('plant', 0.7239134907722473), ('required', 0.7171631455421448)]

cogesa1.1.pdf
create the complex specification for explosive substances and/or articles
['customer', 'underlie', 'article', 'research', 'address', 'create', 'satisfaction', 'success', 'solution', 'constraint', 'feasibility', 'interest', 'substance', 'confidentiality', 'confirm', 'affect', 'improvement', 'adaptation', 'manager', 'criterion']
[('particular', 0.7937981486320496), ('change', 0.7895011901855469), ('example', 0.7835838794708252), ('this', 0.7800945043563843), ('fact', 0.7716935873031616), ('certain', 0.7714309692382812), ('terms', 0.7700669765472412), ('that', 0.7676703929901123), ('specific', 0.7643022537231445), ('need', 0.7584383487701416)]

In [23]:
new_top_terms_dict['semae3068.pdf']

['installation',
 'flight',
 'aircraft',
 'guidance',
 'fastener',
 'gyro',
 'secure',
 'fly',
 'aviation',
 'autopilot',
 'director',
 'instal',
 'install',
 'earth',
 'damage',
 'align',
 'load',
 'position',
 'socket']

In [36]:
counter = 0
for ix in range(len(df_nos_select)):
    nos = df_nos_select.index[ix]
    tmp_sim = model.similar_by_vector(we_p_st[nos])
    tmp_sim = [t[0] for t in tmp_sim]
    if ix<100:
        continue
    if set(tmp_sim).intersection(set(new_top_terms_dict[nos])):
        print(df_nos_select.loc[nos]['NOS Title'])
        counter +=1
    if counter>30:
        break
    #print(nos)
    ##print(df_nos_select.loc[nos]['NOS Title'])
    #print(new_top_terms_dict[nos])
    #print()
    #print()

organise and supervise the delivery of specified targets search plans
provide advice on munition clearance and search related matters
evaluate the effectiveness of specified targets search operations
provide advice on specified targets search matters
confirm the presence of munition(s)
detect and locate specified targets and other risks
contribute to locating and gaining access to specified targets
make presentations on explosives matters
prepare and care for equipment in an explosives environment
manage explosives safety
manage your own resources 
develop your competence in working with munitions or specified targets
manage your own resources and professional development 
manage continuous improvement in activities for explosive substances and articles
allocate and check work in your team 
provide leadership for your team
provide leadership in your area of responsibility 
conduct an assessment of the risks in the workplace
develop the organizational safety policy and/or strategy for e

### Perform hierarchical clustering on each super-suite separately

In [ ]:
sns.set_style("whitegrid")

In [ ]:

# super-suites:
SELECT_MODES = ['engineering','management','construction','financialservices']

SAVEHC = False
STRATEGY = 'tfidf' #'we' or 'tfidf'
WARD = 'a'
for SELECT_MODE in SELECT_MODES[:1]:
    df_nos_n, final_groups, larger_suites, cluster_name, cluster_name_save, \
                cluster_name_figs = select_subdf(SELECT_MODE, 1, 
                                                 1, df_nos_select)
    print('Computing clusters for {}'.format(cluster_name_figs))

    # remove legacy nos
    print('nb with legacy nos: ',len(df_nos_n))
    df_nos_n = df_nos_n[df_nos_n['NOS Title'].map(lambda x: 'legacy' not in x)]
    print('nb without legacy nos 1: ',len(df_nos_n))
    df_nos_n = df_nos_n[df_nos_n.index.map(lambda x: not x[-5:]=='l.pdf')]
    print('nb without legacy nos 2: ',len(df_nos_n))
    suites_in_clus = {}
    groups_clus = df_nos_n.groupby('One_suite')
    for name, group in groups_clus:
        suites_in_clus[name] = list(group['NOS Title'].values)

    if STRATEGY == 'tfidf':
        # this is to get the restricted corpus (to transform, not for fitting)
        textfortoken = df_nos_n['pruned_lemmas']
        tfidfm_n = tfidf_n.transform(textfortoken)
    elif STRATEGY == 'we':
        tfidfm_n = df_nos_n[all_we_cols].values
    else:
        raise ValueError

    # get labels
    if paramsn['bywhich'] == 'suites':
        standard_labels_n = list(df_nos_n.groupby('One_suite').groups.keys())
    else:
        standard_labels_n = list(df_nos_n['NOS Title'].values)

    for ix,t in enumerate(standard_labels_n):
        if len(t)>500:
            # manual correction because of pdf extraction
            standard_labels_n[ix] = standard_labels_n[ix][:50]

    # check best features in a few NOS
    if STRATEGY == 'tfidf':
        for s_idx in range(1): #34):
            s_idx = 0#standard_labels_n.index(
                #'lift and move permanent way materials, components and equipment')
            TF= tfidfm_n[s_idx,:].T.todense()
            print(standard_labels_n[s_idx])
            ix = np.argsort(TF, axis = 0)
            for i in ix[-20:][::-1]: #enumerate(feature_names_n):
                i = np.array(i)
                print(feature_names_n[i[0][0]],np.around(TF[i[0][0]][0,0],3))
            print()


    # perform hierarchical clustering
    distances_n, linkage_matrix_n, c_n, _ = do_hierarch_clustering(tfidfm_n,
                                                                   method = 'average',
                                                                   metric = 'cosine',
                                                                   DOPLOTS= False)


    # Plotting the distance between successive clusters: is there a knee?
    z = linkage_matrix_n[::-1,2]
    knee = np.diff(z, 2)
    
    #fig = plt.figure(figsize = (6,6))
    fig, ax1 = plt.subplots(figsize = (12,6))

    ax2 = ax1.twinx()
    ax1.plot(range(1, len(z)+1), z, 'g-')
    ax2.plot(range(2, len(linkage_matrix_n)), knee, 'b-')
    plt.xlim([0,500])
    ax1.set_ylabel('cluster distance', color='g')
    ax2.set_ylabel('derivative', color='b')

    #plt.plot(range(1, len(z)+1), z)
    #plt.plot(range(2, len(linkage_matrix_n)), knee)
    #plt.xlabel('partition')
    #plt.ylabel('cluster distance')
    plt.title(cluster_name_figs)
    goodness = []
    for i in range(3,100): #len(z)-2):
        lr = LinearRegression(normalize = True)
        lr = lr.fit(np.arange(1,i+1).reshape(-1, 1), z[:i].reshape(-1, 1))
        a1 = lr.score(np.arange(1,i+1).reshape(-1, 1), z[:i].reshape(-1, 1))
        lr = LinearRegression(normalize = True)
        lr = lr.fit(np.arange(i, len(z)).reshape(-1, 1), z[i:].reshape(-1, 1))
        a2 = lr.score(np.arange(i, len(z)).reshape(-1, 1), z[i:].reshape(-1, 1))
        goodness.append(np.around(a1 + a2,4))
        #a1 = scipy.stats.linregress(range(1,i+1), z[:i])
        #a2 = scipy.stats.linregress(range(i, len(z)), z[i:])
        #goodness.append(np.around(a1[2]**2 + a2[2]**2,4))
    
    plt.figure(figsize = (12,6))
    #print(goodness)
    plt.plot(np.arange(3,100), goodness)#len(z)-2),goodness)
    plt.title(cluster_name_figs)
    ixg = np.array(goodness).argmax()+3
    print('best t-point: ',ixg)

    num_ideal = np.ceil(len(df_nos_n)/10)
    print('The ideal number of clusters would be: ',num_ideal)
    num_clust1 = knee.argmax() + 2
    knee[knee.argmax()] = 0
    num_clust2 = knee.argmax() + 2
    
    if SELECT_MODE == 'engineering':
        num_clust = 50 #170 #ixg #max([num_clust1,num_clust2]) #clusters2use[SELECT_MODE][2]
    else:
        if num_clust1 == 2:
            num_clust = num_clust2 #2000
        elif num_clust2 == 2:
            num_clust = num_clust1 #2000
        else:
            num_clust = min([num_clust1,num_clust2])
            
    print('The two peaks are, in order: ',num_clust1, num_clust2)
    print('The selected num clust is ',num_clust)
    #num_clust = max([num_clust1,num_clust2])

    for t in np.arange(0,1,0.05):
        labels_n = fcluster(linkage_matrix_n, t, criterion='distance')
        n_clust = len(set(labels_n))
        if n_clust <= num_clust:
            cutting_th_n = t
            break
    print('cutting threshold: {}'.format(cutting_th_n))       
    
    #Plot the dendrogram (cutting at threshold)
    #cutting_th_n = 0.6
    h = .05*len(df_nos_n)
    fig, ax = plt.subplots(figsize=(28, h)) # set size
    ax = dendrogram(linkage_matrix_n, 
                    labels = [t.capitalize() for t in standard_labels_n], 
                    orientation = 'right', 
                    leaf_font_size=6,
                   color_threshold = cutting_th_n,
                   truncate_mode = 'level', p =15)#,
                   #above_threshold_color = 'k');

    plt.tick_params(axis= 'y',
                    labelsize = 24)
    plt.title('Hierarchical clustering for {}'.format(cluster_name_figs), fontsize = 30)
#              'Hierarchical Clustering Dendrogram of Selected NOS', fontsize = 20)
    plt.xlabel('Distance', fontsize = 30)
    plt.ylabel('NOS title',fontsize = 30)
    
    plt.tight_layout()
    if SAVEHC:
        plt.savefig(os.path.join(output_dir, 
                                 'all_nos_cut_dendrogram_in_{}_{}_{}_{}_{}.png'.format(
            cluster_name_save,qualifier,paramsn['ngrams'],STRATEGY,WARD)), 
                    bbox_inches = "tight")   
        
    # now get and save the clusters
    labels_n = fcluster(linkage_matrix_n, cutting_th_n, criterion='distance')
    short_df_n = df_nos_n.reset_index()[['index','NOS Title', 'One_suite','supersuite']]

    short_df_n['hierarchical'] = labels_n
    short_df_n = short_df_n.set_index('index')
    if SAVEHC:
        short_df_n.to_csv(os.path.join(output_dir, 
                                 'all_nos_cut_labels_in_{}_{}_{}_{}_{}.csv'.format(
            cluster_name_save,qualifier,paramsn['ngrams'],STRATEGY,WARD)))
        
    # print the result of the cut dendrogram
    hierarchical_dict= {}
    L = {}
    D = {}
    for ic in range(1,num_clust+1):
        tmp_local = short_df_n['NOS Title'][
            short_df_n['hierarchical']==ic].values
        if len(tmp_local)<3:
            continue
        hierarchical_dict['{}'.format(ic)] = list(tmp_local)
        A = distance.squareform(distances_n)[(short_df_n['hierarchical']==ic).values,:][:,
                            (short_df_n['hierarchical']==ic).values]
        if A.sum()>0:
            A = np.triu(A)
            A = A[A[:]>0]
        else:
            A = np.ones(1)
        D['{}'.format(ic)] = np.around(np.mean(A),3)
        L['{}'.format(ic)] = (short_df_n['hierarchical']==ic).sum()
    L = pd.DataFrame.from_dict(L, orient = 'index', columns = ['lenght'])
    D = pd.DataFrame.from_dict(D, orient = 'index', columns = ['avg dist'])
    L = L.join(D)
    if SAVEHC:
        L.join(pd.DataFrame.from_dict(hierarchical_dict, orient = 'index')).sort_values(
            by = 'avg dist', ascending = True).T.to_csv(output_dir +
                                '/all_nos_cut_clusters_in_{}_{}_{}_{}_{}.csv'.format(
                                cluster_name_save,qualifier,paramsn['ngrams'],STRATEGY,WARD))

    print()


In [ ]:
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.metrics import adjusted_rand_score

def do_kmean(xx, ks=np.arange(2,4), N=100, N2=100):
        stab = []
        for k in ks:
            t0 = time.time()
            # do N iterations
            stab0 = []
            A = np.empty((xx.shape[0],N))
            for i in range(N):
                k_clus = KMeans(k, n_init = 1, random_state = np.random.randint(1e7))
                A[:,i] = k_clus.fit_predict(xx)
                for j in range(i):
                    stab0.append(adjusted_rand_score(A[:,i],A[:,j]))
            # get stability of clusters for this nb of clusters
            stab.append(np.mean(stab0))
            print_elapsed(t0,'kmeans for k={}'.format(k))
        # what number of clusters has highest stability?
        kmax = ks[np.array(stab).argmax()]
        # redo one last clustering with kmax 
        # and lots of iteration to get the stable versions
        k_clus = KMeans(kmax, n_init= N2)
        labels = k_clus.fit_predict(xx)
        return labels, k_clus, kmax, stab

def get_distance_k(df_row, centroids):
    L = len(df_row.values) # last column is the cluster class
    distance = pairwise_distances(df_row.values[:L-1].reshape(1, -1), 
                                  centroids[int(df_row['k_cluster'])].reshape(1, -1))
    distance = distance[0][0]
    return distance

def get_distance_k2(df_row, labels, centroids):
    Nf = df_row.shape[0] # last column is the cluster class
    N = centroids.shape[0]
    distance = np.empty(N)
    for ix in range(N):
        tmp = df_row[labels == ix]
        tmp2 = pairwise_distances(tmp, centroids[ix].reshape(1,-1))
        distance[ix] = tmp2[0][0]
    #distance = distance[0][0]
    return distance

In [ ]:
SELECT_MODES = ['engineering','management','construction','financialservices']

SAVEKM = True
for SELECT_MODE in SELECT_MODES[:1]:
    df_nos_n, final_groups, larger_suites, cluster_name, cluster_name_save, \
                cluster_name_figs = select_subdf(SELECT_MODE, 1, 
                                                 1,df_nos_select)
    print('Computing clusters for {}'.format(cluster_name_figs))

    # remove legacy nos
    print('nb with legacy nos: ',len(df_nos_n))
    df_nos_n = df_nos_n[df_nos_n['NOS Title'].map(lambda x: 'legacy' not in x)]
    print('nb without legacy nos 1: ',len(df_nos_n))
    df_nos_n = df_nos_n[df_nos_n.index.map(lambda x: not x[-5:]=='l.pdf')]
    print('nb without legacy nos 2: ',len(df_nos_n))
    suites_in_clus = {}
    groups_clus = df_nos_n.groupby('One_suite')
    for name, group in groups_clus:
        suites_in_clus[name] = list(group['NOS Title'].values)

    if STRATEGY == 'tfidf':
        # this is to get the restricted corpus (to transform, not for fitting)
        textfortoken = df_nos_n['pruned_lemmas']
        tfidfm_n = tfidf_n.transform(textfortoken)
    elif STRATEGY == 'we':
        tfidfm_n = df_nos_n[all_we_cols].values
    else:
        raise ValueError

    # get labels
    if paramsn['bywhich'] == 'suites':
        standard_labels_n = list(df_nos_n.groupby('One_suite').groups.keys())
    else:
        standard_labels_n = list(df_nos_n['NOS Title'].values)

    for ix,t in enumerate(standard_labels_n):
        if len(t)>500:
            # manual correction because of pdf extraction
            standard_labels_n[ix] = standard_labels_n[ix][:50]
            
    #
    try:
        print(type(tfidfm_n.toarray()))
        tfidfm_n = tfidfm_n.toarray()#StandardScaler.fit_transform(tfidfm_n.toarray())
        #tfidfm_n = tfidfm_n.toarray()
    except:
        print(type(tfidfm_n))
        tfidfm_n = tfidfm_n
    
    xx = StandardScaler().fit_transform(tfidfm_n)
    out = do_kmean(xx, np.arange(140,221,10), N=100, N2= 100)
    #k_clusters = out[1].labels_.tolist()
    #centroids = out[1].cluster_centers_
    #short_df['k_distance'] = tfidfm_df.apply(get_distance_k, axis =1)
    short_df_n = df_nos_n.reset_index()[['index','NOS Title', 'One_suite','supersuite']]
    short_df_n['kmeans'] = out[0]
    short_df_n = short_df_n.set_index('index')
    if SAVEKM or True:
        short_df_n.to_csv(os.path.join(output_dir, 
                             'all_nos_cut_labels_kmeans_in_{}_{}_{}_{}.csv'.format(
                cluster_name_save,qualifier,paramsn['ngrams'],STRATEGY)))
    
    print(out[2],out[3])
    # get the clusters
    labels2 = np.empty(len(short_df_n), dtype = np.int)
    for ix in range(len(short_df_n)):
        labels2[ix] = int(short_df_n['kmeans'].iloc[ix])
    out_d = get_distance_k2(xx, labels2, out[1].cluster_centers_)
    
    ## collect the centroids (that is, the suite closest to the centroid) 
    ## and print the result of the clustering
    #kmeans_dict= {}
    #most_central = []
    #igroup = 0
    #for name, group in short_df.groupby('k_means'):
    #    kmeans_dict['{}'.format(name)] = group['Suite_names'].values
    #    if igroup < 10:
    #        print(name, group.sort_values(by = 'k_distance').head(3))
    #    igroup += 1
    #    most_central.append(group.sort_values(by = 'k_distance').head(1))
    #if SAVEKM:
    #    pd.DataFrame.from_dict(kmeans_dict, orient = 'index').to_csv(output_dir +
    #                            '/Kmeans_results_{}_{}_{}.csv'.format(qualifier,bywhich,mode))
    
    # print the result of the cut dendrogram
    cluster_dict= {}
    L = {}
    D = {}
    N = max(short_df_n['kmeans'].values)
    print(N)
    for ic in range(N+1):
        tmp_local = short_df_n['NOS Title'][short_df_n['kmeans']==ic].values
        if len(tmp_local)<3:
            continue
        cluster_dict['{}'.format(ic)] = list(tmp_local)
        #A = distance.squareform(distances_n)[(short_df_n['hierarchical']==ic).values,:][:,
        #                    (short_df_n['hierarchical']==ic).values]
        #if A.sum()>0:
        #    A = np.triu(A)
        #    A = A[A[:]>0]
        #else:
        #    A = np.ones(1)
        D['{}'.format(ic)] = out_d[ic]#np.around(np.mean(A),3)
        L['{}'.format(ic)] = (short_df_n['kmeans']==ic).sum()
    L = pd.DataFrame.from_dict(L, orient = 'index', columns = ['lenght'])
    D = pd.DataFrame.from_dict(D, orient = 'index', columns = ['avg dist'])
    L = L.join(D)
    if SAVEKM or True:
        L.join(pd.DataFrame.from_dict(cluster_dict, orient = 'index')).sort_values(
            by = 'avg dist', ascending = True).T.to_csv(output_dir +
                                '/all_nos_cut_clusters_kmeans_in_{}_{}_{}_{}.csv'.format(
                                cluster_name_save,qualifier,paramsn['ngrams'],STRATEGY))
        

In [ ]:
output_dir

In [ ]:
#tmp =pd.read_csv(os.path.join(output_dir, 
#                                 'all_nos_cut_labels_in_{}_{}_{}_v2.csv'.format(
#            cluster_name_save,qualifier,paramsn['ngrams'])))
#tmp2 = pd.read_csv(os.path.join(output_dir, 
#                             'all_nos_cut_labels_kmeans_in_{}_{}_{}.csv'.format(
#        cluster_name_save,qualifier,paramsn['ngrams'])))

In [ ]:
#adjusted_rand_score(tmp['hierarchical'].values,tmp2['kmeans'])